In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import DenseNet201

Using TensorFlow backend.


In [2]:
#TOT = 35534421
TOT = 46613580
EPOCHS = 70
size = 64
batchsize = 256
lw = 6
channel = 3
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = False
optimizer = 'adam'
cb = 'raw'

In [3]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [4]:
K.clear_session()
# model = Xception(input_shape=(size, size, channel), weights=None, classes=NCATS)
model = DenseNet201(input_shape=(size, size, 3), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

x shape (34000, 64, 64, 3) x max 255


In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 3) (34000, 340)
Validation array memory 1.56 GB


In [7]:
train_datagen = image_generator_xd(df_path = '../input/train_all.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel, mixup=mixup, center=center)

In [8]:
model_prefix = 'densenet{}_parts_try2_lw{}_balance_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
if channel == 3:
    model_prefix += '_color'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', save_best_only=True, verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=1,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

densenet64_parts_try2_lw6_balance_0_adam_color


In [9]:
# model.load_weights('models/xception71_parts_lw6_balance_0_adam_color.model')

In [10]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/70
2602/2601 [==============================] - 1040s 400ms/step - loss: 1.8954 - categorical_crossentropy: 1.8954 - categorical_accuracy: 0.5525 - top_3_accuracy: 0.7441 - val_loss: 4.3112 - val_categorical_crossentropy: 4.3112 - val_categorical_accuracy: 0.2081 - val_top_3_accuracy: 0.3669

Epoch 00001: val_top_3_accuracy improved from -inf to 0.36691, saving model to ./models/densenet64_parts_try2_lw6_balance_0_adam_color.model
Epoch 2/70
2602/2601 [==============================] - 1006s 387ms/step - loss: 1.3043 - categorical_crossentropy: 1.3043 - categorical_accuracy: 0.6745 - top_3_accuracy: 0.8494 - val_loss: 1.4854 - val_categorical_crossentropy: 1.4854 - val_categorical_accuracy: 0.6314 - val_top_3_accuracy: 0.8214

Epoch 00002: val_top_3_accuracy improved from 0.36691 to 0.82141, saving model to ./models/densenet64_parts_try2_lw6_balance_0_adam_color.model
Epoch 3/70
2602/2601 [==============================] - 1007s 387ms/step - loss: 1.1749 - categorical_crossentr


Epoch 00039: val_top_3_accuracy improved from 0.92959 to 0.92962, saving model to ./models/densenet64_parts_try2_lw6_balance_0_adam_color.model
Epoch 40/70
2602/2601 [==============================] - 1010s 388ms/step - loss: 0.7321 - categorical_crossentropy: 0.7321 - categorical_accuracy: 0.8097 - top_3_accuracy: 0.9293 - val_loss: 0.7310 - val_categorical_crossentropy: 0.7310 - val_categorical_accuracy: 0.8092 - val_top_3_accuracy: 0.9300

Epoch 00040: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00040: val_top_3_accuracy improved from 0.92962 to 0.93000, saving model to ./models/densenet64_parts_try2_lw6_balance_0_adam_color.model
Epoch 41/70
2602/2601 [==============================] - 1009s 388ms/step - loss: 0.7212 - categorical_crossentropy: 0.7212 - categorical_accuracy: 0.8122 - top_3_accuracy: 0.9303 - val_loss: 0.7232 - val_categorical_crossentropy: 0.7232 - val_categorical_accuracy: 0.8116 - val_top_3_accuracy: 0.9305

Epoch 00041: val_top_3_a

In [6]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


x shape (112199, 64, 64, 3) x max 255
(112199, 3) (112199, 64, 64, 3)
Test array memory 5.14 GB


In [12]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [13]:
doodle_predict(model, check_path, x_test)

112199/112199 [==============================] - 82s 728us/step
save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/densenet64_parts_try2_lw6_balance_0_adam_color.csv.7z -m "submit"


In [5]:
model.load_weights('models/densenet64_parts_try2_lw6_balance_0_adam_color.model')

In [7]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)
np.save('../result/pre_densenet64_parts_try2_lw6_balance_0_adam_color.npy', test_predictions)

112199/112199 [==============================] - 63s 566us/step
